In [1]:

import re

def isVariable(x):
    return len(x) == 1 and x.islower() and x.isalpha()

def getAttributes(string):
    expr = '\([^)]+\)'
    matches = re.findall(expr, string)
    return matches

def getPredicates(string):
    expr = '([a-z~]+)\([^&|]+\)'
    return re.findall(expr, string)

In [2]:
class Fact:
    def __init__(self, expression):
        self.expression = expression
        predicate, params = self.splitExpression(expression)
        self.predicate = predicate
        self.params = params
        self.result = any(self.getConstants())
        
    def splitExpression(self, expression):
        predicate = getPredicates(expression)[0]
        params = getAttributes(expression)[0].strip('()').split(',')
        return [predicate, params]
    
    def getResult(self):
        return self.result
    
    def getConstants(self):
        return [None if isVariable(c) else c for c in self.params]
    
    def getVariables(self):
        return [v if isVariable(v) else None for v in self.params]
    
    def substitute(self, constants):
        c = constants.copy()
        f = f"{self.predicate}({','.join([constants.pop(0) if isVariable(p) else p for p in self.params])})"
        return Fact(f)
    
class Implication:
    def __init__(self, expression):
        self.expression = expression
        l = expression.split('=>')
        self.lhs = [Fact(f) for f in l[0].split('&')]
        self.rhs = Fact(l[1])
        
    def evaluate(self, facts):
        constants = {}
        new_lhs = []
        for fact in facts:
            for val in self.lhs:
                if val.predicate == fact.predicate:
                    for i, v in enumerate(val.getVariables()):
                        if v:
                            constants[v] = fact.getConstants()[i]
                    new_lhs.append(fact)
        predicate, attributes = getPredicates(self.rhs.expression)[0], str(getAttributes(self.rhs.expression)[0])
        for key in constants:
            if constants[key]:
                attributes = attributes.replace(key, constants[key])
        expr = f'{predicate}{attributes}'
        return Fact(expr) if len(new_lhs) and all([f.getResult() for f in new_lhs]) else None

class KB:
    def __init__(self):
        self.facts = set()
        self.implications = set()
            
    def tell(self, e):
        if '=>' in e:
            self.implications.add(Implication(e))
        else:
            self.facts.add(Fact(e))
        for i in self.implications:
            res = i.evaluate(self.facts)
            if res:
                self.facts.add(res)

    def query(self, e):
        facts = set([f.expression for f in self.facts])
        i = 1
        print(f'Querying {e}:')
        for f in facts:
            if Fact(f).predicate == Fact(e).predicate:
                print(f'\t{i}. {f}')
                i += 1
    
    def display(self):
        print("All facts: ")
        for i, f in enumerate(set([f.expression for f in self.facts])):
            print(f'\t{i+1}. {f}')

In [5]:
def main():
    kb = KB()
    n = int(input("Enter number of statements in Knowledge Base: "))
    
    for _ in range(n):
        kb.tell(input())
        
    print("Enter Query:")
    query = input()
    kb.query(query)
    
    kb.display()

In [15]:
#Test1
main()

Enter number of statements in Knowledge Base: 4
cat(x)&hungry(x)=>stare(x)
cat(Chop)
hungry(Chop)
cat(Lula)
Enter Query:
stare(x)
Querying stare(x):
	1. stare(Chop)
All facts: 
	1. cat(Chop)
	2. stare(Chop)
	3. cat(Lula)
	4. hungry(Chop)


In [16]:
#Test1
main()

Enter number of statements in Knowledge Base: 8
missile(x)=>weapon(x)
missile(M1)
enemy(x,America)=>hostile(x)
american(West)
enemy(Nono,America)
owns(Nono,M1)
missile(x)&owns(Nono,x)=>sells(West,x,Nono)
american(x)&weapon(y)&sells(x,y,z)&hostile(z)=>criminal(x)
Enter Query:
criminal(x)
Querying criminal(x):
	1. criminal(West)
All facts: 
	1. hostile(Nono)
	2. enemy(Nono,America)
	3. american(West)
	4. sells(West,M1,Nono)
	5. weapon(M1)
	6. criminal(West)
	7. owns(Nono,M1)
	8. missile(M1)
